## 회귀분석 프로젝트
## - 주제: 롤토체스의 순위방어율 예측

### 1. 주제 선정 배경
    - 코로나 바이러스 여파에도 불구하고, 2020년 1/4분기 게임업계 호황
        - 이는 사회적 거리두기에서 비롯한 비대면(Untact) 문화에 수혜를 입은 것임
        - 코로나가 장기화 되는 상황에서 하반기까지 게임 호황이 지속될 전망임
        
    - 롤토체스, 지난 3월에 출시된 모바일 게임으로 출시 직후 앱스토어 인기 앱 1위 차지
        - 라이엇 게임즈가 롤토체스의 국제 대회인 'Galaxies Championship' 개최 발표
        - 상금은 20만 달러로 전세계 10개국 대상으로 대회가 진행될 예정
        - 게임이 하나의 e스포츠화 되고 있으며, 롤토체스는 전략적 게임으로 아이템, 조합 등 게임 전략이 승패를 좌우
###### 게임에서 순위를 방어할 수 있는(1~4위에 속함) 최적의 조합을 예측해보고자 함

### 2. 롤토체스란?
    - 라이엇 게임즈에서 제작 및 배급한 게임으로, 리그오브레전드의 게임모드로 전략적 팀 전투(TFT)이며, 롤토체스라 불림
    - PC 버전 출시 이후 전세계 이용자수 8,000만 명을 기록
    - 8명의 플레이어가 챔피언과 아이템을 전략적으로 조합하여 최후의 1인이 남을 때까지 전투하는 방식임
        - 챔피언: 52명(계열 시너지: 10개 / 직업 시너지: 13개 조합)
        - 아이템: 54개(기본 아이템 9개 + 기본 아이템 조합 45개)
        - 티어(게임의 평균 수준): 9개(챌린저, 그랜드마스터, 마스터, 다이아몬드, 플래티넘, 골드, 실버, 브론즈, 아이언)

### 3. 데이터 소개
    - 출처 : kaggle - https://www.kaggle.com/gyejr95/tft-match-data
    - 티어별 엑셀 데이터(5개, 각 파일별 게임 10000개 정보)
        - TFT_Challenger_MatchData, TFT_Diamond_MatchData, TFT_GrandMaster_MatchData, TFT_Master_MatchData, TFT_Platinum_MatchData
    - columns 설명
        - 1) gameid: 게임 ID
        - 2) gameDuration: 한 게임당 지속시간
        - 3) level: 플레이어 레벨(1~9)
        - 4) lastRound: 최종 라운드
        - 5) Ranked: 게임 순위(1~8위)
        - 6) ingameDuration: 플레이어별 게임 지속시간
        - 7) combination: 시너지 조합
        - 8) champion: 플레이어에 사용된 챔피언 조합
    - 독립변수: level, lastRound, combination, champion, tier
    - 종속변수: Ranked(순위)

In [3]:
df = pd.read_csv("tft-match-data\\TFT_Challenger_MatchData.csv")
df.tail(8)

,gameId,gameDuration,level,lastRound,Ranked,ingameDuration,combination,champion
79991,KR_4357265434,2094.518555,9,38,1,2086.280273,"{'Blaster': 4, 'Chrono': 2, 'Cybernetic': 1, '...","{'Malphite': {'items': [], 'star': 1}, 'Graves..."
79992,KR_4357265434,2094.518555,9,38,2,2086.280273,"{'Blaster': 2, 'Demolitionist': 2, 'Mercenary'...","{'Ziggs': {'items': [], 'star': 2}, 'Yasuo': {..."
79993,KR_4357265434,2094.518555,9,37,3,2031.166260,"{'Blaster': 2, 'Chrono': 1, 'Cybernetic': 6, '...","{'Fiora': {'items': [36], 'star': 2}, 'Leona':..."
79994,KR_4357265434,2094.518555,7,33,4,1838.332764,"{'DarkStar': 2, 'Demolitionist': 2, 'Infiltrat...","{'KhaZix': {'items': [67, 26, 56], 'star': 2},..."
79995,KR_4357265434,2094.518555,8,33,5,1837.548096,"{'Blaster': 1, 'Chrono': 2, 'Cybernetic': 6, '...","{'Fiora': {'items': [35], 'star': 2}, 'Leona':..."
79996,KR_4357265434,2094.518555,9,33,6,1833.472046,"{'Chrono': 2, 'Cybernetic': 1, 'Demolitionist'...","{'TwistedFate': {'items': [], 'star': 2}, 'Mal..."
79997,KR_4357265434,2094.518555,8,31,7,1730.829712,"{'Blaster': 4, 'Chrono': 2, 'Cybernetic': 1, '...","{'Malphite': {'items': [], 'star': 1}, 'Graves..."
79998,KR_4357265434,2094.518555,6,27,8,1480.288086,"{'ManaReaver': 2, 'Protector': 2, 'Set3_Celest...","{'Darius': {'items': [14, 33, 36], 'star': 2},..."




### 4. 가설 설정
    1) 자주 사용하는 조합일수록 순위방어율이 높을 것이다.
    2) 코스트가 낮은 챔피언을 사용할수록 순위방어율이 낮을 것이다.
    3) 게임시간에 따라 자주 사용되는 조합이 달라질 것이다.